In [ ]:
import tabula as tb
import pandas as pd
import os
import re

In [ ]:
#reads table from pdf file
df = tb.read_pdf("../votes/22-Regular/22-Regular_HM008HVOTE.PDF", pages=1) #address of pdf file

In [ ]:
df[0]

In [ ]:
bool(re.search('HVOTE', 'HB0002HVOTE.pdf'))

In [ ]:
# format table for output
cols = ['NAME', 'YES', 'NO', 'ABS', 'EXC', 'REC']
dfleft = df[0].iloc[:,:6]
dfleft.columns = cols
dfright = df[0].iloc[:,6:]
dfright.columns = cols
fulldf = pd.concat([dfleft, dfright], axis = 0)
fulldf.dropna(subset = ['NAME'], inplace = True)
fulldf.fillna('')

In [ ]:
def PDFTable2DataFrame(file):
    #print(file)
    
    
    if int(file[9:11]) < 21: 
        house_cols = ['NAME', 'YES', 'NO', 'EXC', 'ABS']
        senate_cols = ['NAME', 'YES', 'NO', 'ABS', 'EXC', 'REC']
    else:
        house_cols = ['NAME', 'YES', 'NO', 'PNV', 'EXC', 'ABS']
        senate_cols = ['NAME', 'YES', 'NO', 'ABS', 'EXC', 'REC']
    if bool(re.search('SVOTE', file)) | bool(re.search('SCONCUR', file)) :
        cols = senate_cols
        ncols = len(senate_cols)
    elif bool(re.search('HVOTE', file)) | bool(re.search('HCONCUR', file)): 
        cols = house_cols
        ncols = len(house_cols)
    else: 
        return ""
        
    df = tb.read_pdf(file, pages = 1, silent = True)        
    dfleft = df[0].iloc[:,:ncols]
    dfleft.columns = cols
    dfright = df[0].iloc[:,ncols:]
    dfright.columns = cols
    fulldf = pd.concat([dfleft, dfright], axis = 0)
    fulldf.dropna(subset = ['NAME'], inplace = True)
    fulldf = fulldf.fillna('')
    
    return fulldf

In [ ]:
sessions = os.listdir('../votes/')
#print(vote_files)
for session in sessions[4:]:
    if not os.path.exists(f'../votes-csvs/{session}'):
            os.mkdir(f'../votes-csvs/{session}')
    vote_files = os.listdir(f'../votes/{session}')
    for file in vote_files:
        pdf_path = f'../votes/{session}/{file}'
        try:
            df = PDFTable2DataFrame(pdf_path)
        except:
            with open(f'../votes-csvs/failed-pdfs.log\n', 'a') as f:
                f.write(pdf_path) 
            continue 
        if not isinstance(df, pd.DataFrame): 
            with open(f'../votes-csvs/failed-pdfs.log\n', 'a') as f:
                f.write(pdf_path)
            continue
        file = file[:-4]
        print(file)
        df.to_csv(f'../votes-csvs/{session}/{file}.csv', index = False)

In [ ]:
file = '16-Special_HB0006HVOTE.PDF'
file.find('_')